## **CHAPTER 2 JAX의 특징**

1장에서 JAX를 공부하기 위해 필요한 몇 가지 내용을 간단하게 살펴보았다.
이제 2장에서는 JAX의 더 깊이 있는 기능들을 탐구할 것이다.
`jax.numpy`, JIT 컴파일, 의사 난수 생성, pytree, pmap, JAX의 상태 관리 등에 대해 자세히 다룰 것이다.
이 각각의 기능은 JAX가 제공하는 강력한 도구들로, 고성능 수치 계산과 머신러닝 분야에서 중요한 역할을 한다.

`jax.numpy`는 NumPy의 기능을 JAX로 확장한 것으로, GPU나 TPU와 같은 가속기를 사용하여 높은 성능을 제공한다.
이는 과학 계산과 데이터 분석에 널리 사용되는 표준 도구인 NumPy의 기능을 활용하면서도, JAX의 자동 미분과 벡터화된 연산 능력을 통해 향상된 성능을 경험할 수 있게 한다.
JIT 컴파일은 JAX의 중요한 특징 중 하나로, 파이썬 코드를 고성능의 기계어로 변환하여 실행 속도를 대폭 향상시킨다.
이러한 변환은 특히 복잡한 수치 계산이나 대규모 데이터 처리에서 중요한 성능 향상을 가져온다.

의사 난수 생성은, JAX에서는 재현 가능한 난수 생성을 위한 도구로 사용된다.
pytree는 다양한 데이터 구조를 효율적으로 처리할 수 있게 해주는 유용한 메커니즘이다.
pmap은 병렬처리를 위한 기능ㅇ로, 여러 장치에서 데이터를 분산처리하는 데 사용된다.
마지막으로 JAX의 상태 처리는 함수형 프로그래밍 패러다임을 따르며, 부수 효과와 상태 변경을 최소화하여 프로그램의 예측 가능성과 재사용성을 높인다.

### **2.1 NumPy에서부터 JAX 시작하기**
---

JAX를 설명하기에 앞서, NumPy로 머신러닝을 구현한다고 하면 어떻게 될까?
JAX는 머신러닝을 위해 NumPy를 개선한 프레임워크이기 때문에 이 고민은 의미가 있다.
먼저, NumPy는 기본적으로 배열 데이터를 효율적으로 처리하는 데 탁월한 라이브러리이다,
과학 계산에 널리 사용되며, 머신러닝에서도 데이터 전처리, 통계적 분석 등에 활용되고 있다.
하지만 NumPy는 기본적으로 CPU에서 작동하며 GPU나 TPU 등의 하드웨어 가속에 한계가 있다.
또한 복잡한 미분 연산이나 최적화 작업을 직접 구현하는 것이 번거롭고 어렵다.

이러한 문제를 해결하기 위해 딥마인드는 JAX를 개발했다.
JAX는 기본적으로 NumPy API를 따르지만, 자동 미분(Autograd)과 XLA 컴파일러를 사용하여 GPU 및 TPU에서도 빠르게 작동하도록 설계되었다.
이를 통해 NumPy와 같은 직관적인 인터페이스를 유지하면서도, 머신러닝 연구에서 요구되는 고성능 계산과 복잡한 연산의 자동화를 가능하게 했다.
따라서 JAX는 NumPy를 기반으로 머신러닝을 구현하고자 하는 고민에 대한 혁신적인 해결책을 제시한다.

#### **2.1.1 JAX와 NumPy 비교하기**

먼저, JAX와 NumPy 모듀 파이썬에서 사용할 수 있는 라이브러리이므로, 사용법이 매우 유사하다.
간단한 벡터 연산으로 2개의 라이브러리를 비교해보겠다.
NumPy는 최근 2.0 버전이 나왔지만, 예제를 다루기 위해서는 1.x 버전을 사용해야 한다.

NumPy에서는 다음과 같이 벡터를 생성하고 연산을 수행할 수 있다.

In [1]:
import jax
import jax.numpy as jnp
import numpy as np
from IPython.display import display

x = np.array([1.0, 2.0, 3.0, 4.0])
y = np.array([5.0, 6.0, 7.0, 8.0])
z = x + y

display(z)

array([ 6.,  8., 10., 12.])

마찬가지로, JAX에서도 거의 동일한 방식으로 코드를 작성할 수 있다.

In [2]:
x = jnp.array([1.0, 2.0, 3.0, 4.0])
y = jnp.array([5.0, 6.0, 7.0, 8.0])
z = x + y

display(z)

Array([ 6.,  8., 10., 12.], dtype=float32)

보다시피 JAX는 NumPy와 거의 동일한 API를 제공하므로, NumPy에 익숙한 사용자라면 JAX를 빠르게 습득하고 사용할 수 있다.
또한 JAX의 큰 장점 중 하나는 이런 간단한 연산뿐만 아니라 복잡한 미분 연산 등도 지원한다는 것이다.
이를 통해 머신러닝 모델의 학습과 최적화 과정을 좀 더 쉽게 구현할 수 있다.

#### **2.1.2 JAX에서 미분 계산하기**

JAX의 강력한 기능 중 하나는 자동 미분(automatic differentiation)이다.
이는 머신러닝에서 중요한 역할을 하는데, 모델의 매개변수에 대한 손실 함수(loss function)의 그라디언트(기울기gradient)를 계산하는 데 사용되었기 때문이다.
JAX는 `jax.grad` 함수를 사용하여 이를 간단하게 수행할 수 있다.

f(x) = x^2라는 간단한 함수를 정의하고 그 함수의 미분을 계산해보겠다.

In [3]:
def f(x):
    return x ** 2

df = jax.grad(f)

display(df(3.0))

Array(6., dtype=float32, weak_type=True)

이 코드에서 `jax.grad(f)`는 함수 `f`의 그라디언트를 계산하는 새로운 함수를 반환한다.
이 변환된 함수를 `df`로 정의한다.
따라서 `df(3.0)`은 `f`의 `3.0`에서의 그라디언트, 즉 `2 * x`에 `x = 3.0`을 대입한 결과인 `6`을 반환한다.

이렇게 `jax.grad`를 사용하면 복잡한 함수에 대해서도 그라디언트를 쉽게 계산할 수 있으며, 이는 머신러닝에서 중요한 역할을 하는 경사 하강법(gradient descent) 알고리즘을 구현하는 데 매우 유용하다.
이 외에도 JAX는 미분의 미분인 고계 미분(higher-order derivative) 계산도 `jax.grad`를 여러 번 호출하는 방식으로 매우 간단하게 수행할 수 있다.
예컨대 앞서 정의한 f(x) = x^2 함수의 2차 도함수를 계산해보자.

In [4]:
def f(x):
    return x ** 2

d2f = jax.grad(jax.grad(f))

display(d2f(3.0))

Array(2., dtype=float32, weak_type=True)

이 코드에서 `jax.grad(jax.grad(f))`는 함수 `f`의 2차 도함수를 계산하는 새로운 함수를 반환한다.
즉, `d2f(3.0)`은 `f`의 `3.0`에서의 2차 도함수, `2`를 반환한다.

이처럼 JAX의 자동 미분 기능은 복잡한 미분 연산을 간단하게 처리하며, 이는 딥러닝과 같은 복잡한 머신러닝 모델에서 매개변수를 최적화하는 데 필수이다.
이 기능은 JAX가 다른 수치 계산 라이브러리와 비교했을 때 강력한 도구로 자리 잡게 된 주요 이유 중 하나이다.

#### **2.1.3 손실 함수의 그라디언트 계산하기**

이번엔 예측값과 실젯값을 받아서 손실 함수로 그라디언트를 구해본다.
간단한 손실 함수를 정의하고, 이 함수의 그라디언트를 계산해본다.
이 예제에서는 평균 제곱오차(mean squared error; MSE)를 손실 함수로 사용한다.
이 함수는 회귀 문제에서 자주 사용되는 손실 함수이다.

In [5]:
# 임의의 예측값과 실젯값
y_true = jnp.array([1.0, 2.0, 3.0])
y_pred = jnp.array([1.5, 1.5, 3.5])

# 손실 함수의 그라디언트 계산을 위한 함수 정의
grad_mse_loss = jax.grad(mse_loss, argnums=1)

# 그라디언트 계산
gradient = grad_mse_loss(y_true, y_pred)

display(gradient)

NameError: name 'mse_loss' is not defined

여기서 `jax.grad` 함수의 `argnums` 매개변수는 그라디언트를 계산할 인수의 인덱스를 지정한다.
이 경우 `y_pred`에 대한 그라디언트를 계산하기를 원하므로 `argnums=1`로 설정했다.

이처럼 JAX를 사용하면 손실 함수의 그라디언트를 쉽게 계산할 수 있으며, 이는 모델의 매개변수를 업데이트하는 데 사용할 수 있다.
또한 경사 하강법 등의 그라디언트 기반 최적화 알고리즘을 구현하는 데 매우 유용하다.

추가적으로 `jax.value_and_grad`라는 함수를 사용하면 방금 살펴본 손실 함수의 그라디언트와 동시에 값도 계산할 수 있다.
앞에서 정의한 평균제곱오차 손실 함수에 대해 `jax.value_and_grad`를 사용하면 다음과 같이 프로그래밍할 수 있다.

In [6]:
def mse_loss(y_true, y_pred):
    return jnp.mean((y_true - y_pred) ** 2)

# 임의의 예측값과 실젯값
y_true = jnp.array([1.0, 2.0, 3.0])
y_pred = jnp.array([1.5, 1.5, 3.5])

# 손실 함수와 그라디언트를 동시에 계산하기 위한 함수 정의
value_and_grad_mse_loss = jax.value_and_grad(mse_loss, argnums=1)

# 손실값과 그라디언트 동시에 계산
loss_value, gradient = value_and_grad_mse_loss(y_true, y_pred)

display('Loss Value:', loss_value)
display('Gradient:', gradient)

'Loss Value:'

Array(0.25, dtype=float32)

'Gradient:'

Array([ 0.33333334, -0.33333334,  0.33333334], dtype=float32)